In [1]:
import paho.mqtt.client as mqtt 
# Use json_tricks instead of standard json module to handle serialization
# of numpy arrays
import json_tricks as jt

from opendrop_proxy import SerialProxy


def on_command_recieved(client, userdata, message):
    '''
    Callback function to handle mqtt messages. Note that this being called
    from the paho client event loop. Not sure how this is implemented
    (i.e., is it happening in a background thread?). There are likely
    thread safetey implications.
    '''
    
    payload_str = message.payload.decode("utf-8")
    print("received message: " , str(payload_str))
    payload = jt.loads(payload_str)
    
    if 'args' not in payload:
        payload['args'] = []
    if 'type' not in payload:
        payload['type'] = 'call'

    if payload['type'] == 'call':
        f = getattr(bridge.proxy, payload['command'])
        result = f(*payload['args'])
    elif payload['type'] == 'property':
        if not payload['args']:
            result = getattr(bridge.proxy, payload['command'])
        else:
            setattr(bridge.proxy, payload['command'], payload['args'][0])
            result = None

    if result is not None:
        print("result:", result)
        bridge._client.publish("opendrop/response",
                               jt.dumps({'return': result}))


class MQTTSerialBridge():
    def __init__(self, broker, port):
        '''
        Initialize an MQTTSerialBridge object.

        Parameters
        ----------
        broker : string
            Url for the MQTT broker.
        port : string
            Serial port name (e.g., 'COM1' or '/dev/ttyUSB0')
        '''
        self._broker = broker
        self._client = mqtt.Client("OpenDrop Serial Bridge")
        self._client.connect(broker)
        self.proxy = SerialProxy(port)
        
        self._client.loop_start()
        self._client.subscribe("opendrop/command")
        self._client.on_message=on_command_recieved


bridge = MQTTSerialBridge("test.mosquitto.org", 'COM32')

received message:  {"command": "identify"}
result: GaudiLabs,OpenDrop,#00,v3.2
received message:  {"command": "voltage", "args": [200], "type": "property"}
received message:  {"command": "voltage", "type": "property"}
result: 200.0
received message:  {"command": "set_state_of_channels", "args": [{"__ndarray__": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "dtype": "float64", "shape": [128]}]}
receive